
# Análise Estratégica: Dengue e Clima no Brasil
**Projeto Sistema Dengue-Clima**

Este notebook realiza uma análise aprofundada da relação entre casos de Dengue e variáveis climáticas (temperatura e precipitação), utilizando a camada Gold do Data Lakehouse.

**Objetivos:**
1. Analisar tendências temporais de Dengue e Clima.
2. Identificar correlações estatísticas, incluindo defasagens temporais (lags).
3. Mapear regiões críticas e padrões de surto.
4. Fornecer insights visuais para tomada de decisão governamental.


In [ ]:

import duckdb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
from pathlib import Path

# Configuração de Estilo
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)


## 1. Carregamento e Preparação dos Dados

In [ ]:

# Definir caminho do arquivo Gold
base_path = Path('../data/gold')
gold_file = base_path / 'dengue_clima.parquet'

print(f"Lendo arquivo: {gold_file}")

# Conectar ao DuckDB e ler os dados
con = duckdb.connect(database=':memory:')
df = con.execute(f"SELECT * FROM read_parquet('{gold_file}')").df()

# Converter semana_epidemiologica para datetime (aproximado) para plotagem
# Formato YYYYWW -> Data (segunda-feira da semana)
df['ano'] = df['semana_epidemiologica'].astype(str).str[:4].astype(int)
df['semana'] = df['semana_epidemiologica'].astype(str).str[4:].astype(int)

# Função auxiliar para converter ano-semana em data
def get_start_date(row):
    try:
        return pd.to_datetime(f'{row.ano}-W{row.semana:02d}-1', format='%Y-W%W-%w')
    except:
        return pd.NaT

df['data_inicio_semana'] = df.apply(get_start_date, axis=1)

print(f"Dimensões do Dataset: {df.shape}")
df.head()


## 2. Visão Geral e Qualidade dos Dados

In [ ]:

# Verificar tipos e nulos
df.info()

# Verificar distribuição de nulos nas variáveis climáticas
print("\nPorcentagem de Nulos por Coluna:")
print((df.isnull().sum() / len(df) * 100).round(2))


## 3. Análise Temporal: Casos vs Clima

In [ ]:

# Agregar dados por data para ver a tendência nacional
weekly_stats = df.groupby('data_inicio_semana').agg({
    'casos_notificados': 'sum',
    'inmet_temp_media': 'mean',
    'inmet_precip_tot': 'mean'
}).reset_index()

# Remover últimas semanas se houver queda brusca por atraso de notificação (opcional, análise visual)
weekly_stats = weekly_stats.sort_values('data_inicio_semana')

# Plot: Casos vs Temperatura (FIX: Usar Matplotlib direto para evitar conflito de tipos)
fig, ax1 = plt.subplots(figsize=(14, 7))

# Gráfico de barras (casos)
color = 'tab:red'
ax1.set_xlabel('Semana Epidemiológica')
ax1.set_ylabel('Casos Notificados', color=color)
# Barras no eixo categórico (índice numérico)
bar_positions = np.arange(len(weekly_stats))
bars = ax1.bar(bar_positions, weekly_stats['casos_notificados'], color=color, alpha=0.3)
ax1.tick_params(axis='y', labelcolor=color)

# Gráfico de linha (temperatura) no segundo eixo
ax2 = ax1.twinx()
color = 'tab:orange'
ax2.set_ylabel('Temp. Média (°C)', color=color)
# Linha no eixo numérico (índice numérico)
line = ax2.plot(bar_positions, weekly_stats['inmet_temp_media'], color=color, marker='o', linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)

# Ajustar labels do eixo x para mostrar apenas algumas datas
n = 4  # Mostrar cada enésimo label
ax1.set_xticks(bar_positions[::n])
ax1.set_xticklabels(weekly_stats['data_inicio_semana'].dt.strftime('%Y-%m-%d')[::n], rotation=45, ha='right')

plt.title('Evolução Temporal: Casos de Dengue vs Temperatura Média (Nacional)', fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:

# Plot: Casos vs Precipitação (FIX: Usar Matplotlib direto)
fig, ax1 = plt.subplots(figsize=(14, 7))

# Gráfico de barras (casos)
color = 'tab:red'
ax1.set_xlabel('Semana Epidemiológica')
ax1.set_ylabel('Casos Notificados', color=color)
bar_positions = np.arange(len(weekly_stats))
bars = ax1.bar(bar_positions, weekly_stats['casos_notificados'], color=color, alpha=0.3)
ax1.tick_params(axis='y', labelcolor=color)

# Gráfico de linha (precipitação) no segundo eixo
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Precipitação Total (mm)', color=color)
line = ax2.plot(bar_positions, weekly_stats['inmet_precip_tot'], color=color, marker='o', linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)

# Ajustar labels do eixo x
ax1.set_xticks(bar_positions[::4])
ax1.set_xticklabels(weekly_stats['data_inicio_semana'].dt.strftime('%Y-%m-%d')[::4], rotation=45, ha='right')

plt.title('Evolução Temporal: Casos de Dengue vs Precipitação (Nacional)', fontsize=16)
plt.tight_layout()
plt.show()


## 4. Análise de Correlação e Lags


Investigamos a correlação entre casos de dengue e variáveis climáticas com defasagem (lags) de 1 a 4 semanas, pois o ciclo do mosquito e incubação do vírus levam tempo.


In [ ]:

# Selecionar colunas numéricas relevantes
cols_corr = [
    'casos_notificados',
    'inmet_temp_media', 'inmet_precip_tot',
    'inmet_temp_media_lag1', 'inmet_temp_media_lag2', 'inmet_temp_media_lag3', 'inmet_temp_media_lag4',
    'inmet_precip_tot_lag1', 'inmet_precip_tot_lag2', 'inmet_precip_tot_lag3', 'inmet_precip_tot_lag4'
]

# Calcular matriz de correlação (Spearman é melhor para relações não lineares/assimetricas)
corr_matrix = df[cols_corr].corr(method='spearman')

# Plotar Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix[['casos_notificados']].sort_values(by='casos_notificados', ascending=False), 
            annot=True, cmap='coolwarm', vmin=-1, vmax=1, linewidths=.5)
plt.title('Correlação de Spearman com Casos de Dengue (com Lags)', fontsize=14)
plt.show()


## 5. Análise Regional: Focos de Atenção

In [ ]:

# Agrupar por Município (Top 10 em casos totais)
top_cities = df.groupby(['uf', 'nome_municipio'])['casos_notificados'].sum().sort_values(ascending=False).head(10).reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x='casos_notificados', y='nome_municipio', hue='uf', data=top_cities, dodge=False)
plt.title('Top 10 Municípios com Mais Casos Notificados (Período Analisado)', fontsize=14)
plt.xlabel('Total de Casos')
plt.ylabel('Município')
plt.show()


In [ ]:

# Análise de Dispersão: Temperatura Média vs Casos (para municípios com > 100 casos)
df_high_cases = df[df['casos_notificados'] > 100]

plt.figure(figsize=(10, 6))
sns.scatterplot(x='inmet_temp_media_lag2', y='casos_notificados', data=df_high_cases, hue='uf', alpha=0.6)
plt.title('Relação: Temperatura (Lag 2 sem) vs Casos (Municípios com surtos)', fontsize=14)
plt.xlabel('Temp. Média (Lag 2 semanas) °C')
plt.ylabel('Casos Notificados')
plt.yscale('log') # Escala logarítmica para visualizar melhor a distribuição
plt.grid(True, which="both", ls="-", alpha=0.2)
plt.show()



## 6. Conclusões e Insights para Gestão

Com base nas análises gráficas e estatísticas:

1. **Sazonalidade:** Observa-se claramente a relação sazonal entre o aumento das chuvas/temperatura e o pico de casos de Dengue.
2. **Tempo de Resposta (Lag):** A análise de correlação indica qual defasagem (1-4 semanas) tem maior impacto. Geralmente, espera-se que as condições climáticas de 2 a 4 semanas atrás influenciem os casos atuais (ciclo do vetor).
3. **Alertas Regionais:** O gráfico de dispersão e o ranking de municípios ajudam a identificar onde os surtos são mais intensos e se estão associados a temperaturas específicas (ex: faixa ótima de 25-30°C).

**Próximos Passos Sugeridos:**
*   Implementar um modelo preditivo (Random Forest ou ARIMA) usando as variáveis de lag identificadas.
*   Criar análises interativas para monitoramento em tempo real.